In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 63kB/s 
     |████████████████████████████████| 204kB 45.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=f9a4b311fd3f02d01629ac4445382f669907e84adb3cf1b7df4f865d29a2fef8
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Primr Cuatrimestre de 2016. Examen parcial, segunda oportunidad: 

### 1) Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón. (*) (15 pts)

In [6]:
# (chat_id, user_id, nickname, text)
chats  =[(1000,200,'Juan','dasdasd?dasdas?dasdas??asdasdasd'),
         (1000,201,'Barto','?asdasdasdasdasd'),
         (1000,202,'Lala96','asdsdasdasdasd'),
         (1000,203,'BeluKittie','asdadasd??asdad?asdasdasdasd?asdasd??asdasd'),
         (1000,204,'Tomas','dasd?asd??asdasdasd'),
         (1000,205,'AsadoDeRemolacha','asdasd?'),
         (1000,200,'Juan','asdasd?asd?'),
         (1000,201,'Barto','asdasd'),
         (1000,202,'Lala96','asdasdd'),
         (1000,203,'BeluKittie','asdasd?asdasd?'),
         (1000,204,'Tomas','asdasd?'),
         (1000,205,'AsadoDeRemolacha','asdasd???asdasd?'),
         (1001,200,'Juan','asdasd?asdasd?'),
         (1001,201,'Barto','asd??asdasd?'),
         (1001,202,'Lala96','asd?asd?'),
         (1001,203,'BeluKittie','asdasd'),
         (1001,204,'Tomas','asdasd'),
         (1001,205,'AsadoDeRemolacha','asdasd'),
         (1001,200,'Juan','asdasd'),
         (1001,201,'Barto','asdasd'),
         (1001,202,'Lala96','asdasd'),
         (1001,203,'BeluKittie','asd?asd'),
         (1001,204,'Tomas','?'),
         (1001,205,'AsadoDeRemolacha','asdasd?'),
         (1001,200,'Juan','asdasd'),
         (1001,201,'Barto','asd?asd?asd?')


         ]

In [7]:
chatsRDD = sc.parallelize(chats)
chatsRDD.collect()

[(1000, 200, 'Juan', 'dasdasd?dasdas?dasdas??asdasdasd'),
 (1000, 201, 'Barto', '?asdasdasdasdasd'),
 (1000, 202, 'Lala96', 'asdsdasdasdasd'),
 (1000, 203, 'BeluKittie', 'asdadasd??asdad?asdasdasdasd?asdasd??asdasd'),
 (1000, 204, 'Tomas', 'dasd?asd??asdasdasd'),
 (1000, 205, 'AsadoDeRemolacha', 'asdasd?'),
 (1000, 200, 'Juan', 'asdasd?asd?'),
 (1000, 201, 'Barto', 'asdasd'),
 (1000, 202, 'Lala96', 'asdasdd'),
 (1000, 203, 'BeluKittie', 'asdasd?asdasd?'),
 (1000, 204, 'Tomas', 'asdasd?'),
 (1000, 205, 'AsadoDeRemolacha', 'asdasd???asdasd?'),
 (1001, 200, 'Juan', 'asdasd?asdasd?'),
 (1001, 201, 'Barto', 'asd??asdasd?'),
 (1001, 202, 'Lala96', 'asd?asd?'),
 (1001, 203, 'BeluKittie', 'asdasd'),
 (1001, 204, 'Tomas', 'asdasd'),
 (1001, 205, 'AsadoDeRemolacha', 'asdasd'),
 (1001, 200, 'Juan', 'asdasd'),
 (1001, 201, 'Barto', 'asdasd'),
 (1001, 202, 'Lala96', 'asdasd'),
 (1001, 203, 'BeluKittie', 'asd?asd'),
 (1001, 204, 'Tomas', '?'),
 (1001, 205, 'AsadoDeRemolacha', 'asdasd?'),
 (1001, 200

In [11]:
#funcion que identifica las preguntas
def get_pregunta (s):
  lista_aux = []
  lista_aux = s.split('?')
  return (len(lista_aux)-1)



2

In [20]:
preguntas_por_usuario = ((chatsRDD.map(lambda x: (x[1],get_pregunta(x[3])))).reduceByKey(lambda x,y: x+y)).reduce(lambda x,y: x if x[1]>y[1] else y)
preguntas_por_usuario

(203, 9)